In [17]:
from bluesky import RunEngine
from bluesky import Msg
from bluesky.plan_stubs import *
from IPython import get_ipython
user_ns = get_ipython().user_ns
from bluesky.preprocessors import SupplementalData

RE = RunEngine({})

from bluesky.callbacks.best_effort import BestEffortCallback
bec = BestEffortCallback()

# Send all metadata/data captured to the BestEffortCallback.
RE.subscribe(bec)

#Get the databroker https://gist.github.com/untzag/53281819709b1058c7708a250cbd3676
from databroker.v2 import temp
db = temp()

# Insert all metadata/data captured into db.
RE.subscribe(db.v1.insert)

from tqdm import tqdm
from bluesky.utils import ProgressBarManager, ProgressBar

#from bluesky.utils.jupyter import pbar_manager_for_notebook
#RE.waiting_hook = pbar_manager_for_notebook()

#ProgressBarManager(delay_draw=0.1)
#RE.subscribe(watch)

#RE.waiting_hook = ProgressBarManager()

#RE.waiting_hook = ProgressBa
#sd = SupplementalData()
#RE.preprocessors.append(sd)


In [18]:
help(ProgressBar)

Help on class ProgressBar in module bluesky.utils:

class ProgressBar(builtins.object)
 |  ProgressBar(status_objs, delay_draw=0.2)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, status_objs, delay_draw=0.2)
 |      Represent status objects with a progress bars.
 |      
 |      Parameters
 |      ----------
 |      status_objs : list
 |          Status objects
 |      delay_draw : float, optional
 |          To avoid flashing progress bars that will complete quickly after
 |          they are displayed, delay drawing until the progress bar has been
 |          around for awhile. Default is 0.2 seconds.
 |  
 |  clear(self)
 |  
 |  draw(self)
 |  
 |  update(self, pos, *, name=None, current=None, initial=None, target=None, unit='units', precision=None, fraction=None, time_elapsed=None, time_remaining=None)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variable

In [19]:
import sys
sys.path.append('..')

In [20]:
from ophyd import PVPositioner, EpicsSignal, EpicsSignalRO, Device, DerivedSignal,PseudoPositioner,PseudoSingle
from ophyd import Component as Cpt
from ophyd.pseudopos import (pseudo_position_argument,
                             real_position_argument)
from ophyd.signal import Signal, SignalRO
from ophyd import FormattedComponent as FCpt



#https://github.com/pcdshub/pcdsdevices/blob/master/pcdsdevices/signal.py
class InternalSignal(SignalRO):
    """
    Class Signal that stores info but should only be updated by the class.
    SignalRO can be updated with _readback, but this does not process
    callbacks. For the signal to behave normally, we need to bypass the put
    override.
    To put to one of these signals, simply call put with force=True
    """

    def put(self, value, *, timestamp=None, force=False):
        return Signal.put(self, value, timestamp=timestamp, force=force)

    def set(self, value, *, timestamp=None, force=False):
        return Signal.set(self, value, timestamp=timestamp, force=force)

# based on https://github.com/pcdshub/pcdsdevices/blob/master/pcdsdevices/pv_positioner.py with readback included
class PVPositionerDone(PVPositioner):
    """
    PV Positioner with no readback that reports done immediately.
    Optionally, this PV positioner can be configured to skip small moves,
    e.g. moves that are smaller than the atol value.
    Parameters
    ----------
    prefix: str
        PV prefix for the request setpoint. This should always be a hutch name.
    name: str, required keyword
        Name to use for this device in log messages, data streams, etc.
    skip_small_moves: bool, optional
        Defaults to False. If True, ignores move requests that are smaller
        than the atol factor.
        This can be very useful for synchronized energy scans where the ACR
        side of the process can be very slow, but does not necessarily need to
        happen at every step. Rather than design complicated scan patterns, we
        can skip the small moves here and move the monochromater and beam
        request devices in parallel.
    
    """

    # Override setpoint, readback in subclass
    setpoint = None
    readback = None

    done = Cpt(InternalSignal, value=0)
    done_value = 1

    
    def _setup_move(self, position):
       
        super()._setup_move(position)
        self._toggle_done()

    def _toggle_done(self):
        self.done.put(0, force=True)
        self.done.put(1, force=True)
        
        

class PVPositionerComparator(PVPositioner):
    """
    PV Positioner with a software done signal.
    The done state is set by a comparison function defined in the class body.
    The comparison function takes two arguments, readback and setpoint,
    returning True if we are close enough to be considered done or False if we
    are too far away.
    """

    # Override setpoint, readback in subclass
    setpoint = None
    readback = None

    done = Cpt(InternalSignal, value=0)
    done_value = 1

    # Optionally override limits to a 2-element tuple in subclass
    limits = None

    def __init__(self, prefix, *, name, **kwargs):
        self._last_readback = None
        self._last_setpoint = None
        super().__init__(prefix, name=name, **kwargs)
        if None in (self.setpoint, self.readback):
            raise NotImplementedError('PVPositionerComparator requires both '
                                      'a setpoint and a readback signal to '
                                      'compare!')

    def done_comparator(self, readback, setpoint):
        """Override done_comparator in subclass."""
        raise NotImplementedError('Must implement a done comparator!')

    def __init_subclass__(cls, **kwargs):
        """Set up callbacks in subclass."""
        super().__init_subclass__(**kwargs)
        if None not in (cls.setpoint, cls.readback):
            cls.setpoint.sub_value(cls._update_setpoint)
            cls.readback.sub_value(cls._update_readback)

    def _update_setpoint(self, *args, value, **kwargs):
        """Callback to cache the setpoint and update done state."""
        self._last_setpoint = value
        # Always set done to False when a move is requested
        # This means we always get a rising edge when finished moving
        # Even if the move distance is under our done moving tolerance
        self.done.put(0, force=True)
        self._update_done()

    def _update_readback(self, *args, value, **kwargs):
        """Callback to cache the readback and update done state."""
        self._last_readback = value
        self._update_done()

    def _update_done(self):
        """Update our status to done if we pass the comparator."""
        if None not in (self._last_readback, self._last_setpoint):
            is_done = self.done_comparator(self._last_readback,
                                           self._last_setpoint)
            self.done.put(int(is_done), force=True)

In [21]:
from ophyd import PVPositioner, EpicsSignal, EpicsSignalRO, Device, DerivedSignal,PseudoPositioner,PseudoSingle,EpicsMotor
from ophyd import Component as Cpt

from ophyd import FormattedComponent as FCpt
from ophyd.pseudopos import (pseudo_position_argument,
                             real_position_argument)
from ophyd.signal import Signal, SignalRO

In [22]:

class Axis(PVPositionerComparator):

    setpoint = FCpt(EpicsSignal,'{self.prefix}set{self._ch_name}') # component is itself a class
    readback = FCpt(EpicsSignal,'{self.prefix}get{self._ch_name}')
            
    atol = 0.001  # tolerance before we set done to be 1

    def done_comparator(self, readback, setpoint):
        return setpoint-self.atol < readback < setpoint+self.atol
        
        
    def __init__(self, prefix, ch_name=None, **kwargs):
        self._ch_name = ch_name
        super().__init__(prefix, **kwargs)

class Robot(Device):

    axis_1 = Cpt(Axis, 'axis1:',ch_name='Pos')
    axis_2 = Cpt(Axis, 'axis2:',ch_name='Pos')
    #axis_3 = Cpt(Axis, 'axis3:')

In [23]:
dummy_axis = Robot('EMILEL:TestIOC00:',name='dummy_axis')

In [24]:
dummy_axis.axis_1.readback.get()

0.1

In [25]:
Status = dummy_axis

In [26]:
Status

Robot(prefix='EMILEL:TestIOC00:', name='dummy_axis', read_attrs=['axis_1', 'axis_1.done', 'axis_1.setpoint', 'axis_1.readback', 'axis_2', 'axis_2.done', 'axis_2.setpoint', 'axis_2.readback'], configuration_attrs=['axis_1', 'axis_2'])

In [27]:
dummy_axis.axis_1.done

InternalSignal(name='dummy_axis_axis_1_done', parent='dummy_axis_axis_1', value=1, timestamp=1626866099.5260372)

In [28]:
(dummy_axis.axis_1.setpoint.set(1))

Status(obj=EpicsSignal(read_pv='EMILEL:TestIOC00:axis1:setPos', name='dummy_axis_axis_1_setpoint', parent='dummy_axis_axis_1', value=0.1, timestamp=1626452816.254218, auto_monitor=False, string=False, write_pv='EMILEL:TestIOC00:axis1:setPos', limits=False, put_complete=False), done=False, success=False)

In [29]:
def move_to(axis_1_target, axis_2_target):
    start_pos1 = dummy_axis.axis_1.readback.get()
    start_pos2 = dummy_axis.axis_2.readback.get()
    #start_pos3 = dummy_axis.axis_3.readback.get()
    print(f'Initial position of axis1: {start_pos1}. \n Target position of axis1: {axis_1_target}. \n')
    print(f'Initial position of axis2: {start_pos2}. \n Target position of axis2: {axis_2_target}. \n')
    #print(f'Initial position of axis3: {start_pos3}. \n Target position of axis3: {axis_3_target}. \n')
    
    yield from mv(dummy_axis.axis_1, axis_1_target,
                  dummy_axis.axis_2, axis_2_target)
                  #dummy_axis.axis_3, axis_3_target)
    final_pos1 = dummy_axis.axis_1.readback.get()
    final_pos2 = dummy_axis.axis_2.readback.get()
    #final_pos3 = dummy_axis.axis_3.readback.get()
    
    final_done1 = dummy_axis.axis_1.done.get()
    final_done2 = dummy_axis.axis_2.done.get()
    #final_done3 = dummy_axis.axis_3.done.get()
    
    if (final_done1 == 1) and (round(axis_1_target, 2) == round(final_pos1, 2)):
        print(f'Final position of axis1: {final_pos1}. DONE!')
    else:
        print(f'Final position of axis1 {final_pos1} is not identical to the target position {axis_1_target}. ERROR!')                      
                              #yield from mv(dummy_axis.axis_1, 4)
    if (final_done2 == 1) and (round(axis_2_target, 2) == round(final_pos2, 2)):
        print(f'Final position of axis1: {final_pos2}. DONE!')
    else:
        print(f'Final position of axis1 {final_pos2} is not identical to the target position {axis_2_target}. ERROR!')                      
                              #yield from mv(dummy_axis.axis_1, 4)
            
    #if (final_done3 == 1) and (round(axis_3_target, 2) == round(final_pos3, 2)):
    #    print(f'Final position of axis1: {final_pos3}. DONE!')
    #else:
    #    print(f'Final position of axis1 {final_pos3} is not identical to the target position {axis_3_target}. ERROR!')                      
                              #yield from mv(dummy_axis.axis_1, 4)
    
    

In [30]:
list_append = []

#dict_in = {'dummy_axis' : {'axis_1' : ('target', 'vx_set', 'vx_read'), 
#                           'axis_2' : ('target', 'vy_set', 'vy_read')} 
                  
    
def get_intial_pos(dict_in, list_readback_in, list_target_in):
    device = list(dict_in.keys())
    components = list(dict_in[device[0]].keys())
    #value_of_component = list(dict_in[device[0]].values())
    for i in components:        
        create_string_for_getattr_readback = f'{i}.readback'
        readback_get_full = getattr(user_ns[device[0]], create_string_for_getattr_readback)
        readback_get_value = readback_get_full.get()
        list_readback_in.append(readback_get_value) 
        create_string_for_target_value = f'{i}'
        target_value = dict_in[device[0]][create_string_for_target_value]
        list_target_in.append(target_value)
        print(f'Initial position of {i}: {float(readback_get_value)}. Target position of {i}: {float(target_value)}.')
        
    print('\n')
    
def get_final_pos(dict_in, list_readback_in, list_target_in, deviation_in):
    
    def deviation_range(input_value, target_value, deviation):
        target_value_plus = target_value + deviation  
        target_value_minus = target_value - deviation
        if (input_value <= target_value_plus) and (input_value >= target_value_minus):
            status = True
        else: 
            status = False
        return status

    device = list(dict_in.keys())
    components = list(dict_in[device[0]].keys())
    #value_of_component = list(dict_in[device[0]].values())
    list_final_pos_in = []
    list_final_done_in = []
    
    for i in components:        
        create_string_for_getattr_readback = f'{i}.readback'
        readback_get_full = getattr(user_ns[device[0]], create_string_for_getattr_readback)
        readback_get_value = readback_get_full.get()
        list_final_pos_in.append(readback_get_value) 
        
        create_string_for_getattr_readback_done = f'{i}.done'
        readback_done_get_full = getattr(user_ns[device[0]], create_string_for_getattr_readback_done)
        readback_done_get_value = readback_done_get_full.get()
        list_final_done_in.append(readback_done_get_value)
    
   
    
    # initial, target, final
    tuple_final_values = list(zip(list_final_done_in, list_target_in, list_final_pos_in))
    for entries in tuple_final_values:
        #print(f'{deviation_range(round(float(entries[2]), 2), round(float(entries[1]), 2), deviation_in)}')
        #if (entries[0] == 1) and (round(float(entries[1]), 2) == round(float(entries[2]), 2)):
        if (entries[0] == 1) and (deviation_range(round(float(entries[2]), 2), round(float(entries[1]), 2), deviation_in)):
            print(f'Final position of axis1: {float(entries[2])}. DONE!')
        else:
            print(f'Final position of axis1 {float(entries[2])} is not identical to the target position {float(entries[1])}. ERROR!')     

In [31]:
dict_in = {'dummy_axis' : {'axis_1' : '0.2', 
                           'axis_2' : '3'}} 
device = list(dict_in.keys())
list_readback = []
list_target = []
deviation = 0.1
get_intial_pos(dict_in, list_readback, list_target)
get_final_pos(dict_in, list_readback, list_target, deviation)

#print(list_append)



Initial position of axis_1: 1.0. Target position of axis_1: 0.2.
Initial position of axis_2: 3.0. Target position of axis_2: 3.0.


Final position of axis1 1.0 is not identical to the target position 0.2. ERROR!
Final position of axis1: 3.0. DONE!


In [195]:
dummy_axis.axis_1.readback.get()


1.0

In [196]:
dummy_axis.axis_2.readback.get()

0.5

In [197]:
dummy_axis.axis_2.setpoint.set(0.5)

Status(obj=EpicsSignal(read_pv='EMILEL:TestIOC00:axis2:setPos', name='dummy_axis_axis_2_setpoint', parent='dummy_axis_axis_2', value=0.5, timestamp=1626449774.50487, auto_monitor=False, string=False, write_pv='EMILEL:TestIOC00:axis2:setPos', limits=False, put_complete=False), done=False, success=False)

In [198]:
dummy_axis.axis_1.readback.get()

1.0

In [199]:
dict_in = {'dummy_axis' : {'axis_1' : '0.1', 
                               'axis_2' : '3'}} 
device = list(dict_in.keys())    
components = list(dict_in[list(dict_in.keys())[0]].keys())

In [200]:

getattr(user_ns[device[0]],f'{components[0]}')

Axis(prefix='EMILEL:TestIOC00:axis1:', name='dummy_axis_axis_1', parent='dummy_axis', settle_time=0.0, timeout=None, read_attrs=['done', 'setpoint', 'readback'], configuration_attrs=[], limits=None, egu='')

In [201]:
#msg.dummy_axis.read(*msg.args, **msg.kwargs)
RE.state 

'idle'

In [202]:

def test_msgs(RE, hw):
    m = Msg('read', hw)
    print(m.command)
    print(m.obj)
    print(m.args)
    print(m.kwargs)
    print(m.obj.name)

In [203]:
test_msgs(RE, dummy_axis)

read
Robot(prefix='EMILEL:TestIOC00:', name='dummy_axis', read_attrs=['axis_1', 'axis_1.done', 'axis_1.setpoint', 'axis_1.readback', 'axis_2', 'axis_2.done', 'axis_2.setpoint', 'axis_2.readback'], configuration_attrs=['axis_1', 'axis_2'])
()
{}
dummy_axis


In [204]:
getattr(user_ns[device[0]],f'{components[0]}')

Axis(prefix='EMILEL:TestIOC00:axis1:', name='dummy_axis_axis_1', parent='dummy_axis', settle_time=0.0, timeout=None, read_attrs=['done', 'setpoint', 'readback'], configuration_attrs=[], limits=None, egu='')

In [205]:
RE(Msg('read', getattr(user_ns[device[0]],f'{components[0]}')))

()

In [206]:
# the Msg command is useless for obtaining information of the current run status....

def test_msgs1(RE, hw):
    m0 = Msg('open_run',hw,1)
    print(m0)
    m = Msg('set', hw.axis_1, 2)
    print(m)
    print(m.args)
    m2 = Msg('read', hw.axis_1)
    print(m2.obj.read_attrs)
    m3 = Msg('close_run',hw)
    print(m3)
test_msgs1(RE, dummy_axis)

Msg('open_run', obj=Robot(prefix='EMILEL:TestIOC00:', name='dummy_axis', read_attrs=['axis_1', 'axis_1.done', 'axis_1.setpoint', 'axis_1.readback', 'axis_2', 'axis_2.done', 'axis_2.setpoint', 'axis_2.readback'], configuration_attrs=['axis_1', 'axis_2']), args=(1,), kwargs={}, run=None)
Msg('set', obj=Axis(prefix='EMILEL:TestIOC00:axis1:', name='dummy_axis_axis_1', parent='dummy_axis', settle_time=0.0, timeout=None, read_attrs=['done', 'setpoint', 'readback'], configuration_attrs=[], limits=None, egu=''), args=(2,), kwargs={}, run=None)
(2,)
['done', 'setpoint', 'readback']
Msg('close_run', obj=Robot(prefix='EMILEL:TestIOC00:', name='dummy_axis', read_attrs=['axis_1', 'axis_1.done', 'axis_1.setpoint', 'axis_1.readback', 'axis_2', 'axis_2.done', 'axis_2.setpoint', 'axis_2.readback'], configuration_attrs=['axis_1', 'axis_2']), args=(), kwargs={}, run=None)


In [207]:

def test_list_of_msgs(RE, hw):
    # smoke tests checking that RunEngine accepts a plain list of Messages
    RE([Msg('open_run'), Msg('set', hw.axis_1, 1), Msg('close_run')])

In [208]:
#test_list_of_msgs(RE, dummy_axis)

In [227]:
def test_tuple_progress():

    class StatusPlaceholder:
        "Just enough to make ProgressBar happy. We will update manually."
        def __init__(self):
            self.done = False

        def watch(self, _):
            ...
            

    # where the progress bar computes the fraction
    st = StatusPlaceholder()
    pbar = ProgressBar([st])
    pbar.update(0, name='',
                current=(0, 0), initial=(0, 0), target=(1, 1))
    pbar.update(0, name='',
                current=(0.2, 0.2), initial=(0, 0), target=(1, 1))
    pbar.update(0, name='',
                current=(1, 1), initial=(0, 0), target=(1, 1))
    st.done = True
    pbar.update(0, name='',
                current=(1, 1), initial=(0, 0), target=(1, 1))
    
"""
    # where the status object computes the fraction
    st = StatusPlaceholder()
    pbar = ProgressBar([st])
    pbar.update(0, name='',
                current=(0, 0), initial=(0, 0), target=(1, 1),
                fraction=0)
    pbar.update(0, name='',
                current=(0.2, 0.2), initial=(0, 0), target=(1, 1),
                fraction=0.2)
    pbar.update(0, name='',
                current=(1, 1), initial=(0, 0), target=(1, 1),
                fraction=1)
    st.done = True
    pbar.update(0, name='',
                current=(1, 1), initial=(0, 0), target=(1, 1),
                fraction=1)
"""


  
"""
    # minimal API
    st = StatusPlaceholder()
    pbar = ProgressBar([st])
    pbar.update(0)
    pbar.update(0)
    st.done = True
    pbar.update(0)

    # name only
    st = StatusPlaceholder()
    pbar = ProgressBar([st])
    pbar.update(0, name='foo')
    pbar.update(0, name='foo')
    st.done = True
    pbar.update(0, name='foo')
"""

"\n    # minimal API\n    st = StatusPlaceholder()\n    pbar = ProgressBar([st])\n    pbar.update(0)\n    pbar.update(0)\n    st.done = True\n    pbar.update(0)\n\n    # name only\n    st = StatusPlaceholder()\n    pbar = ProgressBar([st])\n    pbar.update(0, name='foo')\n    pbar.update(0, name='foo')\n    st.done = True\n    pbar.update(0, name='foo')\n"

In [235]:
from bluesky.tests import requires_ophyd
@requires_ophyd
def test_status_with_name(hw):
    from ophyd.status import DeviceStatus
    st = DeviceStatus(device=hw.det)
    pbar = ProgressBar([st])
    st._finished()

    st = DeviceStatus(device=hw.det)
    pbar = ProgressBar([st])
    #assert pbar.delay_draw == 0.2
    time.sleep(0.25)
    st._finished()

ModuleNotFoundError: No module named 'pytest'

In [236]:
def test_draw_before_update():
    class Status:
        done = False
        def watch(self, func):
            ...

    # Test that the default meter placeholder is valid to draw.
    pbar = ProgressBar([Status()])
    pbar.draw()

In [49]:

RE.waiting_hook = ProgressBarManager()        
    
def move_to_general():
    #test_draw_before_update()
    # define the input for function
#    dict_in = {'dummy_axis' : {'axis_1' : '0.1', 
#                               'axis_2' : '3'}} 

    dict_in = {'dummy_axis' : {'axis_2' : '2'}} 
   
    device = list(dict_in.keys()) 
    components = list(dict_in[list(dict_in.keys())[0]].keys())
    list_readback = []
    list_target = []
    deviation = 0.1
    #sd.baseline = [user_ns[device[0]]]
    print('\n')
    print(f'### Moving device: {device[0]} ###\n')
    get_intial_pos(dict_in, list_readback, list_target)
    
    # insert the original function code
    #yield from Msg('open_run') #Msg('create', 'dummy_axis', name='primary')
    yield from mv(getattr(user_ns[device[0]],f'{components[0]}'), float(dict_in[device[0]][components[0]])                  
                 )
    
#    yield from mv(getattr(user_ns[device[0]],f'{components[0]}'), float(dict_in[device[0]][components[0]]),
#                  getattr(user_ns[device[0]],f'{components[1]}'), float(dict_in[device[0]][components[1]])
#                 )
                  #dummy_axis.axis_3, axis_3_target)
      
    get_final_pos(dict_in, list_readback, list_target,deviation)    
    print('\n')

In [50]:
def move_to_general1():
    #test_draw_before_update()
    # define the input for function
#    dict_in = {'dummy_axis' : {'axis_1' : '0.1', 
#                               'axis_2' : '3'}} 
    
    dict_in = {'dummy_axis' : {'axis_1' : '0.9'
                               }} 
    
    device = list(dict_in.keys()) 
    components = list(dict_in[list(dict_in.keys())[0]].keys())
    list_readback = []
    list_target = []
    deviation = 0.1
    #sd.baseline = [user_ns[device[0]]]
    print('\n')
    print(f'### Moving device: {device[0]} ###\n')
    get_intial_pos(dict_in, list_readback, list_target)
    
    # insert the original function code
    #yield from Msg('open_run') #Msg('create', 'dummy_axis', name='primary')
    yield from mv(getattr(user_ns[device[0]],f'{components[0]}'), float(dict_in[device[0]][components[0]])                 
                 )
    
#    yield from mv(getattr(user_ns[device[0]],f'{components[0]}'), float(dict_in[device[0]][components[0]]),
#                  getattr(user_ns[device[0]],f'{components[1]}'), float(dict_in[device[0]][components[1]])
#                 )
                  #dummy_axis.axis_3, axis_3_target)
      
    get_final_pos(dict_in, list_readback, list_target,deviation)    
    print('\n')

In [51]:
RE.waiting_hook.delay_draw == 0.2

True

In [52]:
ProgressBarManager(delay_draw=0.001)

In [53]:
RE(move_to_general())
#RE(sleep(1))
RE(move_to_general1())



### Moving device: dummy_axis ###

Initial position of axis_2: 1.0. Target position of axis_2: 2.0.


dummy_axis_axis_2: 100%|████████████████████▉| 0.999/1.0 [00:00<00:00,  4.91/s]
dummy_axis_axis_2: 100%|███████████████████████| 1.0/1.0 [00:00<00:00,  3.29/s]
dummy_axis_axis_2 [In progress. No progress bar available.]                    
                                                                               
Final position of axis1: 1.9999. DONE!




### Moving device: dummy_axis ###

Initial position of axis_1: 1.0. Target position of axis_1: 0.9.


dummy_axis_axis_1:  99%|████████████████████▊| 0.099/0.1 [00:00<00:00,  1.93s/]
dummy_axis_axis_1: 100%|███████████████████████| 0.1/0.1 [00:00<00:00,  2.91s/]
dummy_axis_axis_1 [In progress. No progress bar available.]                    
                                                                               
Final position of axis1: 0.9001. DONE!




()

In [212]:
RE.md

{'versions': {'ophyd': '1.6.0', 'bluesky': '1.6.7'}}

In [213]:
print(RE.commands)

['create', 'save', 'drop', 'read', 'monitor', 'unmonitor', 'null', 'RE_class', 'stop', 'set', 'trigger', 'sleep', 'wait', 'checkpoint', 'clear_checkpoint', 'rewindable', 'pause', 'resume', 'collect', 'kickoff', 'complete', 'configure', 'stage', 'unstage', 'subscribe', 'unsubscribe', 'open_run', 'close_run', 'wait_for', 'input', 'install_suspender', 'remove_suspender']


In [68]:
#print(RE.print_command_registry(verbose=True))

In [66]:
Msg('create', 'dummy_axis', name='primary')

Msg('create', obj='dummy_axis', args=(), kwargs={'name': 'primary'}, run=None)

In [62]:
 Msg('create', name='primary')


Msg('create', obj=None, args=(), kwargs={'name': 'primary'}, run=None)

In [24]:
status = dummy_axis.axis_1.done.get()

In [25]:
status

1

In [26]:
RE(move_to(1, 0.1))

Initial position of axis1: 0.1. 
 Target position of axis1: 1. 

Initial position of axis2: 2.0. 
 Target position of axis2: 0.1. 

Final position of axis1: 0.99991. DONE!
Final position of axis1: 0.10019. DONE!


()

In [27]:
user_ns['dummy_axis'].axis_1.readback.get()

1.0

In [28]:
RE.stop()

TransitionError: RunEngine is already idle.

In [77]:
round(5.76543, 2)

5.77

In [78]:
round(5.75, 1)

5.8

In [79]:
#from bluesky.utils import jupyter.NotebookProgressBar

In [87]:
from bluesky.magics import BlueskyMagics

get_ipython().register_magics(BlueskyMagics)

In [88]:
%h

UsageError: Line magic function `%h` not found.


In [19]:
    dict_in = {'dummy_axis' : {'axis_1' : '0.1', 
                               'axis_2' : '2'}} 

In [20]:
dict_in


{'dummy_axis': {'axis_1': '0.1', 'axis_2': '2'}}

In [23]:
    dict_in = {'dummy_axis2' : {'axis_1' : '0.1', 
                               'axis_2' : '2'}} 

In [24]:
dict_in

{'dummy_axis2': {'axis_1': '0.1', 'axis_2': '2'}}

In [17]:
dict_in['dummy_axis3'] = {'axis_1': '0.1', 'axis_2': '2'}

In [18]:
dict_in

{'dummy_axis': {'axis_1': '0.1', 'axis_2': '2'},
 'dummy_axis1': {'axis_1': '0.1', 'axis_2': '2'},
 'dummy_axis3': {'axis_1': '0.1', 'axis_2': '2'}}